Let us agggregate data in 5-day intervals. That way we can use the variables whose temporal resolution are of 5-day granularity and also shortens the length of time series. It is likely(?) that we do not need track every thing on a daily basis.

- Dec. 29, 2025
- HN

In [1]:
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import pandas as pd
import numpy as np
import random
import os, os.path, pickle, sys

import matplotlib
import matplotlib.pyplot as plt

from typing import List, Optional, Any

import configparser
import argparse

# from pathlib import Path
# PROJECT_ROOT = Path.cwd().parent
# sys.path.insert(0, str(PROJECT_ROOT))
# from transformersCores import config
# from transformersCores import config as cfg


import transformersCores.config as cfg
import transformersCores.preprocess as prp

In [2]:
import importlib
importlib.reload(cfg);
importlib.reload(prp);

In [3]:
dir_base = "./../../../01_research_data/Ehsan/wheat/"
data_base = dir_base + "Data/"
raw_dir = os.path.join(data_base, "00_raw_by_location/")
configs_dir = dir_base + "configs/"

In [4]:
params = cfg.aggregateParameters()
params.readFromFile(configs_dir + "Scenario_1_aggr_cols.cfg")
# params = vars(params)

In [5]:
# params_v2 = cfg.aggregateParameters()
# params_v2.load_config_from_file(configs_dir + "Scenario_1_aggr_cols.cfg")

### Read Data

In [46]:
filename = raw_dir + "raw_all_locations_separate_daily_monthly_fiveDay.sav"
data_dict = pd.read_pickle(filename)

In [47]:
data_dict.keys()

dict_keys(['wheat_all_locs_raw_daily', 'wheat_all_locs_raw_monthly', 'wheat_all_locs_raw_fiveDay', 'varieties_traits', 'source_code', 'Author', 'Date', 'Note'])

In [48]:
raw_daily = data_dict["wheat_all_locs_raw_daily"]
raw_fiveDay = data_dict["wheat_all_locs_raw_fiveDay"]
varieties_traits = data_dict["varieties_traits"]

In [49]:
raw_daily.head(2)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,final_season,freezing_dd_cday,heat_dd_cday,year,planting_date,harvest_date
0,farmington,2001-10-01,274,0,Emergence,0.0,25.25,7.35,1.41,4.0,...,17.9,0.0,0.0,0,Preseason,0.0,2.46,2001,NaN,NaN
1,farmington,2001-10-02,275,1,Emergence,0.0,21.65,8.05,1.18,3.2,...,13.6,0.0,0.0,0,Preseason,0.0,0.00,2001,NaN,NaN


array([nan], dtype=object)

In [11]:
params = cfg.aggregateParameters()
params.readFromFile(configs_dir + "Scenario_1_aggr_cols.cfg")

daily_converted_2_5day = prp.aggregate_per5day(params = params, daily_df = raw_daily, fiveDay_df = raw_fiveDay)

In [16]:
raw_daily[(raw_daily["location"] == "walla_walla") & (raw_daily["year"] == 2003)].head(5)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,final_season,freezing_dd_cday,heat_dd_cday,year,planting_date,harvest_date,five_day_date
122336,walla_walla,2003-01-01,1,0,Emergence,2.2,7.35,0.55,0.19,0.9,...,0.323529,0.0,1,Preseason,3.68,0.0,2003,2003-03-24,2003-07-25,2003-01-05
122337,walla_walla,2003-01-02,2,1,Emergence,6.1,11.75,5.45,0.29,1.6,...,0.968254,0.0,1,Preseason,0.00,0.0,2003,2003-03-24,2003-07-25,2003-01-05
122338,walla_walla,2003-01-03,3,2,Emergence,2.4,10.95,6.85,0.31,1.3,...,0.585366,0.0,1,Preseason,0.00,0.0,2003,2003-03-24,2003-07-25,2003-01-05
122339,walla_walla,2003-01-04,4,3,Emergence,3.7,10.55,7.15,0.27,1.4,...,1.088235,0.0,1,Preseason,0.00,0.0,2003,2003-03-24,2003-07-25,2003-01-05
122340,walla_walla,2003-01-05,5,4,Emergence,0.0,8.35,2.75,0.24,0.9,...,0.000000,0.0,0,Preseason,1.48,0.0,2003,2003-03-24,2003-07-25,2003-01-05


In [17]:
raw_fiveDay[(raw_fiveDay["date"].dt.year == 2003) & (raw_fiveDay["location"] == "walla_walla")].head(5)

,date,doy,location,std_precip_evap_unitless,palmer_drought_unitless
24543,2003-01-05,5,walla_walla,0.90,-3.53
24544,2003-01-10,10,walla_walla,0.90,-3.45
24545,2003-01-15,15,walla_walla,-0.17,-3.40
24546,2003-01-20,20,walla_walla,-0.80,-3.37
24547,2003-01-25,25,walla_walla,0.03,-3.30


In [18]:
end_date = pd.to_datetime("2003-01-05")
loc = "walla_walla"
# end_date = raw_fiveDay["date"][0]
five_days = pd.date_range(end=end_date, periods=5, freq='D')
a_col = "precip_mmday"

raw_daily[(raw_daily["location"] == loc) & (raw_daily["date"].isin(five_days))][a_col].sum()

np.float64(14.400000000000002)

In [21]:
daily_converted_2_5day[(daily_converted_2_5day["location"] == loc) & \
                       (daily_converted_2_5day["five_day_date"] == end_date)][a_col]

24542    14.4
Name: precip_mmday, dtype: float64

In [31]:
daily_converted_2_5day.head(2)

,location,date,mean_avg_rh_perc,mean_avg_temp_c,mean_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,mean_heat_dd_cday,...,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,final_season,season,stage,planting_date,harvest_date
0,almira,2001-10-02,43.25,15.20,0.0,1.20,0.5,14.00,0.000,0.68,...,0.004335,163.45,1.245,0.0,2.70,Preseason,Preseason,Emergence,NaN,NaN
1,almira,2001-10-07,41.92,9.79,0.0,2.13,4.0,12.96,1.344,0.00,...,0.003190,148.38,0.844,0.0,3.26,Preseason,Preseason,Emergence,NaN,NaN


In [38]:
daily_converted_2_5day[(daily_converted_2_5day["location"] ==loc) & \
                       (daily_converted_2_5day["date"] == end_date)][ "sum_" + a_col]

24542    14.4
Name: sum_precip_mmday, dtype: float64

In [40]:
daily_converted_2_5day[(daily_converted_2_5day["location"] == loc)].head(5)

,location,date,mean_avg_rh_perc,mean_avg_temp_c,mean_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,mean_heat_dd_cday,...,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,final_season,season,stage,planting_date,harvest_date
24462,walla_walla,2001-10-02,38.125,19.25,0.0,0.75,0.5,13.10,0.000,3.01,...,0.005070,180.25,1.620,0.0,3.65,Preseason,Preseason,Emergence,NaN,NaN
24463,walla_walla,2001-10-07,38.760,13.52,0.0,2.24,4.0,12.90,0.016,0.00,...,0.003784,159.70,1.096,0.2,3.66,Preseason,Preseason,Emergence,NaN,NaN
24464,walla_walla,2001-10-12,64.570,9.08,0.0,1.93,9.0,9.02,0.752,0.00,...,0.005072,98.86,0.418,0.8,8.24,Preseason,Preseason,Emergence,NaN,NaN
24465,walla_walla,2001-10-17,63.790,11.34,0.0,2.18,14.0,11.66,0.352,0.00,...,0.005400,127.74,0.598,0.4,6.60,Preseason,Preseason,Emergence,NaN,NaN
24466,walla_walla,2001-10-22,66.420,9.44,0.0,2.08,19.0,10.66,0.832,0.00,...,0.005400,98.60,0.416,0.4,6.32,Preseason,Preseason,Emergence,NaN,NaN


In [45]:
raw_daily[(raw_daily["location"] == "walla_walla") & (raw_daily["year"] == 2001)]["planting_date"].unique()

array([nan], dtype=object)